In [ ]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv' , index_col='id')
df

In [ ]:
X = df.drop(columns=['target'] , axis=1)
y = df['target']

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

selector = SelectKBest(f_classif, k=50)
selected_features = selector.fit_transform(X, y)
f_score_indexes = (-selector.scores_).argsort()

plt.plot(selector.scores_)

In [ ]:
f_score_indexes

In [ ]:

plt.hist(y)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

im = RandomOverSampler(random_state=2)

resampled_X , resampled_y = im.fit_resample(X , y)

In [ ]:
plt.hist(resampled_y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(resampled_X, resampled_y ,test_size=0.2 ,random_state=2)# , stratify=y)
print(X_train.shape , X_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import RidgeClassifier ,Lasso 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
model = RandomForestClassifier(max_depth=13 , random_state=2)
# model = RidgeClassifier(alpha = 1  , random_state=2)
# model = Lasso(alpha=1.0)
# model = LinearSVC(C=0.5)
# model = KNeighborsClassifier(n_neighbors=5)

pass_important_features = ColumnTransformer(
    transformers=[
        ('pass' , 'passthrough' ,  list(f_score_indexes)[:43])
    ] , remainder='drop'
)
mypl = Pipeline(
    steps=[
        ('important' , pass_important_features),
        ('model' , model)
    ]
)
mypl.fit(X_train , y_train)
print('training score = ' , roc_auc_score(y_true =y_train ,y_score=mypl.predict(X_train)) * 100 , '\t' 
 , 'testing score  = ' , roc_auc_score(y_true =y_test  ,y_score=mypl.predict(X_test )) * 100)


In [ ]:
test_df = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv' , index_col='id')
arr = mypl.predict(test_df)
out = pd.DataFrame(arr , index = test_df.index , columns=['target'])
out.to_csv('submission.csv')
out